<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/sd_validator_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das dependências de vetorialização (HuggingFace)
!pip install sentence-transformers -q
print("✅ Motor Vetorial instalado. Carregando modelos neurais...")

In [ ]:
from sentence_transformers import SentenceTransformer, util
import time
import pandas as pd

# --- CONFIGURAÇÃO DO USUÁRIO (EDITE AQUI) ---
name_to_test = "Engenheiro Estoico"  # Ex: O nome curto do seu agente
domain_to_test = "Profissional de tecnologia focado em fundamentos sólidos, resiliente a hype e com clareza técnica." # A missão completa

# --- PARÂMETROS DE RIGOR CIENTÍFICO (SLE v1.1) ---
THRESHOLDS = {
    "sd_pass": 0.70,          # Similaridade mínima aceitável
    "sd_cross_platform": 0.55,# Mínimo para passar em modelos leves
    "minimalism": 4           # Máximo de palavras permitidas no nome
}

# --- ENGINE DE VALIDAÇÃO ---
def load_models():
    # Usamos dois modelos para evitar viés de arquitetura (Cross-Validation)
    return {
        'miniLM': SentenceTransformer('all-MiniLM-L6-v2'),
        'mpnet': SentenceTransformer('all-mpnet-base-v2')
    }

def run_sd_check(name, domain):
    print(f"🔬 Iniciando Auditoria Semântica para: '{name}'...")
    models = load_models()
    results = {}
    scores = []

    # 1. Cálculo Vetorial
    for model_name, model in models.items():
        emb_name = model.encode(name)
        emb_domain = model.encode(domain)
        score = util.cos_sim(emb_name, emb_domain)[0][0].item()
        results[model_name] = score
        scores.append(score)

    # 2. Análise de Métricas
    sd_mean = sum(scores) / len(scores)
    word_count = len(name.split())

    # 3. Veredito (Lógica Booleana)
    pass_density = sd_mean >= THRESHOLDS["sd_pass"]
    pass_minimalism = word_count <= THRESHOLDS["minimalism"]

    overall_status = "PASS" if (pass_density and pass_minimalism) else "FAIL"

    return {
        "status": overall_status,
        "metrics": {
            "mean_density": sd_mean,
            "word_count": word_count,
            "scores": results
        },
        "checks": {
            "density_pass": pass_density,
            "minimalism_pass": pass_minimalism
        }
    }

# --- EXECUÇÃO E RELATÓRIO ---
audit = run_sd_check(name_to_test, domain_to_test)

print("\n" + "="*60)
print(f"📊 RELATÓRIO DE VALIDAÇÃO CIENTÍFICA (SLE)")
print("="*60)

df = pd.DataFrame([audit["metrics"]["scores"]]).T
df.columns = ["Cosine Similarity"]
print(f"\n1. Análise Vetorial (Cross-Model):\n{df}")

print(f"\n2. Verificação de Minimalismo:")
print(f"   - Palavras: {audit['metrics']['word_count']} (Limite: {THRESHOLDS['minimalism']})")
print(f"   - Status: {'✅ OK' if audit['checks']['minimalism_pass'] else '❌ FALHA'}")

print(f"\n3. Verificação de Densidade:")
print(f"   - Média SD: {audit['metrics']['mean_density']:.4f} (Mínimo: {THRESHOLDS['sd_pass']})")
print(f"   - Status: {'✅ OK' if audit['checks']['density_pass'] else '❌ FALHA'}")

print("-" * 60)
if audit["status"] == "PASS":
    print("🏆 RESULTADO FINAL: APROVADO (SCIENTIFIC VALIDATION PASSING)")
    print("   Seu agente possui alta densidade semântica e está pronto para produção.")
else:
    print("⚠️ RESULTADO FINAL: REPROVADO")
    print("   Ajuste o nome para ser mais curto ou mais alinhado semanticamente ao domínio.")
print("-" * 60)